## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [12]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Bluff,NZ,-46.6000,168.3333,54.00,67,51,5.01,broken clouds
1,1,Bengkulu,ID,-3.8004,102.2655,82.11,75,55,1.59,broken clouds
2,2,Atar,MR,20.5169,-13.0499,78.87,23,54,6.02,broken clouds
3,3,Touros,BR,-5.1989,-35.4608,73.40,94,20,4.61,few clouds
4,4,Paita,PE,-5.0892,-81.1144,69.40,77,91,15.61,overcast clouds


In [13]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 32
What is the maximum temperature you would like for your trip? 55


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
new_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                       (city_data_df['Max Temp'] >= min_temp)]
new_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Bluff,NZ,-46.6000,168.3333,54.00,67,51,5.01,broken clouds
7,7,Aklavik,CA,68.2191,-135.0107,50.00,46,100,5.75,overcast clouds
11,11,Ushuaia,AR,-54.8000,-68.3000,42.80,70,20,7.00,few clouds
16,16,Saldanha,ZA,-33.0117,17.9442,53.60,87,0,16.22,clear sky
27,27,Sahy,SK,48.0741,18.9495,55.00,91,100,3.00,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
643,643,Saint-Quentin,FR,49.8333,3.3333,46.99,93,20,8.05,few clouds
648,648,Filimonovo,RU,56.2011,95.4611,45.64,72,100,5.75,overcast clouds
649,649,Inuvik,CA,68.3499,-133.7218,50.00,46,100,5.75,overcast clouds
657,657,Honningsvag,NO,70.9821,25.9704,35.67,80,100,22.50,overcast clouds


In [15]:
# 4a. Determine if there are any empty rows.
new_df.isnull().sum()

City_ID        0
City           0
Country        1
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_df = new_df.dropna()

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Bluff,NZ,54.00,broken clouds,-46.6000,168.3333,
7,Aklavik,CA,50.00,overcast clouds,68.2191,-135.0107,
11,Ushuaia,AR,42.80,few clouds,-54.8000,-68.3000,
16,Saldanha,ZA,53.60,clear sky,-33.0117,17.9442,
27,Sahy,SK,55.00,overcast clouds,48.0741,18.9495,
30,Fortuna,US,55.00,overcast clouds,40.5982,-124.1573,
34,Hinckley,GB,48.99,clear sky,52.5389,-1.3761,
45,Yellowknife,CA,39.20,broken clouds,62.4560,-114.3525,
55,Koppang,NO,42.66,light rain,61.5706,11.0555,
60,Punta Arenas,CL,44.60,light rain,-53.1500,-70.9167,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"     
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Bluff,NZ,54.00,broken clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
7,Aklavik,CA,50.00,overcast clouds,68.2191,-135.0107,Aklavik Inn
11,Ushuaia,AR,42.80,few clouds,-54.8000,-68.3000,Albatros Hotel
16,Saldanha,ZA,53.60,clear sky,-33.0117,17.9442,Blue Bay Lodge
27,Sahy,SK,55.00,overcast clouds,48.0741,18.9495,Tünde Vendégház
...,...,...,...,...,...,...,...
643,Saint-Quentin,FR,46.99,few clouds,49.8333,3.3333,ibis Saint-Quentin Basilique
648,Filimonovo,RU,45.64,overcast clouds,56.2011,95.4611,
649,Inuvik,CA,50.00,overcast clouds,68.3499,-133.7218,Nova Inn Inuvik
657,Honningsvag,NO,35.67,overcast clouds,70.9821,25.9704,Nordkapp Vandrerhjem


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))